# Twitter Sentiment Analysis with NLP
## A Complete Machine Learning Pipeline for Social Media Text Classification

Welcome to this comprehensive tutorial on Twitter sentiment analysis! In this notebook, we'll build a complete Natural Language Processing (NLP) pipeline to analyze and classify sentiments from Twitter data.

### 🎯 Objectives
- Build an end-to-end sentiment analysis system
- Explore and preprocess Twitter data
- Extract meaningful features from text
- Train and compare multiple machine learning models
- Evaluate model performance with comprehensive metrics
- Create a production-ready prediction system

### 🛠 Tools & Technologies
- **Python**: Core programming language
- **Pandas**: Data manipulation and analysis
- **Scikit-Learn**: Machine learning algorithms and evaluation
- **NLTK**: Natural language processing toolkit
- **Matplotlib & Seaborn**: Data visualization
- **NumPy**: Numerical computations

### 📊 Dataset
We'll work with a Twitter dataset containing tweets labeled with three sentiment categories:
- **Positive**: Happy, satisfied, enthusiastic tweets
- **Negative**: Angry, disappointed, critical tweets  
- **Neutral**: Factual, informational, or neutral tweets

Let's get started! 🚀

## 1. Import Required Libraries

First, let's import all the necessary libraries for our sentiment analysis pipeline.

In [ ]:
# Data manipulation and analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Machine learning libraries
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, classification_report, 
                            confusion_matrix, f1_score, precision_score, 
                            recall_score)

# Text processing libraries
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Additional utilities
import warnings
import os
import sys
from datetime import datetime
import json

# Configure settings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Set random seed for reproducibility
np.random.seed(42)

print("✅ All libraries imported successfully!")
print(f"📦 Pandas version: {pd.__version__}")
print(f"📦 NumPy version: {np.__version__}")
print(f"📦 Scikit-learn version: {sklearn.__version__}")

In [ ]:
# Download required NLTK data
try:
    nltk.data.find('tokenizers/punkt')
    nltk.data.find('corpora/stopwords')
    nltk.data.find('corpora/wordnet')
    print("✅ NLTK data already available")
except LookupError:
    print("📥 Downloading NLTK data...")
    nltk.download('punkt', quiet=True)
    nltk.download('stopwords', quiet=True)
    nltk.download('wordnet', quiet=True)
    print("✅ NLTK data downloaded successfully")

## 2. Load and Explore Twitter Dataset

Let's load our Twitter dataset and perform exploratory data analysis to understand the structure and distribution of our data.

In [ ]:
# Create sample Twitter dataset for demonstration
def create_sample_dataset():
    """Create a sample Twitter dataset for sentiment analysis"""
    
    # Sample tweets for each sentiment
    sample_data = {
        'text': [
            # Positive tweets
            "I absolutely love this new movie! Best film of the year! 🎬✨",
            "Amazing product! Exceeded all my expectations. Highly recommend! 👍",
            "Such a beautiful day today! Perfect weather for a walk 🌞",
            "Just got promoted at work! So excited and grateful! 🎉",
            "This restaurant has the most delicious food ever! 🍕",
            "Great customer service! The staff was so helpful 😊",
            "Successfully completed my marathon today! Feeling accomplished! 🏃‍♂️",
            "Love spending time with family. Best weekend ever! ❤️",
            "This book is incredible! Can't put it down 📚",
            "Perfect vacation! Beautiful beaches and sunsets 🏖️",
            
            # Negative tweets  
            "Terrible movie! Complete waste of time and money 😡",
            "Worst customer service ever! Rude staff and long waits 😤",
            "This product is broken and useless. Total disappointment!",
            "Horrible weather today! Rain ruined all my plans ☔",
            "Got stuck in traffic for 2 hours! So frustrating 🚗",
            "This restaurant has terrible food! Overpriced and tasteless",
            "Feeling sick and exhausted. Worst day ever! 😷",
            "Computer crashed and lost all my work! So angry 💻",
            "Cancelled flight ruined vacation plans. Terrible service ✈️",
            "This book is boring and poorly written 📖",
            
            # Neutral tweets
            "The movie was okay. Not great but not terrible either",
            "Weather is average today. Neither sunny nor rainy ⛅",
            "This product works as expected. Nothing special",
            "Had lunch at a new restaurant. Food was decent",
            "Regular day at work. Nothing exciting happened",
            "The book is fine. Some interesting parts 📚",
            "Traffic was normal today. No major delays",
            "This coffee shop is alright. Standard service ☕",
            "Average shopping experience. Found what I needed",
            "The concert was okay. Not the best but decent 🎵"
        ],
        'sentiment': (
            ['positive'] * 10 + 
            ['negative'] * 10 + 
            ['neutral'] * 10
        )
    }
    
    return pd.DataFrame(sample_data)

# Create and load the dataset
df = create_sample_dataset()

print("📊 Dataset Overview:")
print(f"Total number of tweets: {len(df)}")
print(f"Number of columns: {df.shape[1]}")
print(f"Column names: {list(df.columns)}")

# Display first few rows
print("\n📝 Sample tweets:")
df.head()

In [ ]:
# Explore sentiment distribution
print("📈 Sentiment Distribution:")
sentiment_counts = df['sentiment'].value_counts()
print(sentiment_counts)

# Calculate percentages
sentiment_percentages = df['sentiment'].value_counts(normalize=True) * 100
print("\n📊 Sentiment Percentages:")
for sentiment, percentage in sentiment_percentages.items():
    print(f"{sentiment.capitalize()}: {percentage:.1f}%")

# Visualize sentiment distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Bar plot
sentiment_counts.plot(kind='bar', ax=ax1, color=['#ff6b6b', '#4ecdc4', '#45b7d1'])
ax1.set_title('Sentiment Distribution (Count)', fontsize=14, fontweight='bold')
ax1.set_xlabel('Sentiment')
ax1.set_ylabel('Number of Tweets')
ax1.tick_params(axis='x', rotation=45)

# Pie chart
ax2.pie(sentiment_counts.values, labels=sentiment_counts.index, autopct='%1.1f%%',
        colors=['#ff6b6b', '#4ecdc4', '#45b7d1'], startangle=90)
ax2.set_title('Sentiment Distribution (Percentage)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

# Text length analysis
df['text_length'] = df['text'].str.len()
df['word_count'] = df['text'].str.split().str.len()

print("\n📏 Text Length Statistics:")
print(f"Average character length: {df['text_length'].mean():.1f}")
print(f"Average word count: {df['word_count'].mean():.1f}")
print(f"Min length: {df['text_length'].min()}")
print(f"Max length: {df['text_length'].max()}")

## 3. Data Preprocessing and Cleaning

Now let's clean our dataset by handling any data quality issues and preparing it for text processing.

In [ ]:
# Check for data quality issues
print("🔍 Data Quality Check:")
print(f"Missing values in 'text': {df['text'].isnull().sum()}")
print(f"Missing values in 'sentiment': {df['sentiment'].isnull().sum()}")
print(f"Duplicate rows: {df.duplicated().sum()}")
print(f"Empty text entries: {(df['text'].str.strip() == '').sum()}")

# Check for any unexpected sentiment labels
print(f"\nUnique sentiment labels: {df['sentiment'].unique()}")

# Remove any rows with missing or empty text
initial_count = len(df)
df = df.dropna(subset=['text', 'sentiment'])
df = df[df['text'].str.strip() != '']
final_count = len(df)

if initial_count != final_count:
    print(f"⚠️ Removed {initial_count - final_count} rows with missing/empty data")
else:
    print("✅ No missing or empty data found")

# Ensure sentiment labels are standardized
df['sentiment'] = df['sentiment'].str.lower().str.strip()

print(f"\n📊 Clean dataset shape: {df.shape}")
print("✅ Data preprocessing completed successfully!")

## 4. Text Preprocessing for NLP

This is the most crucial step for sentiment analysis! We'll clean and normalize the tweet text to make it suitable for machine learning algorithms.

In [ ]:
def preprocess_text(text):
    """
    Comprehensive text preprocessing function for tweets
    """
    if not isinstance(text, str):
        return ""
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove URLs
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
    
    # Remove @mentions and #hashtags
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#\w+', '', text)
    
    # Remove email addresses
    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '', text)
    
    # Remove extra whitespace, newlines, and tabs
    text = re.sub(r'\s+', ' ', text)
    
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    
    # Remove extra spaces
    text = text.strip()
    
    return text

def advanced_preprocess_text(text):
    """
    Advanced preprocessing with tokenization and stopword removal
    """
    # Apply basic preprocessing first
    text = preprocess_text(text)
    
    if not text:
        return ""
    
    # Tokenize
    tokens = word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words and len(word) > 2]
    
    # Apply lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    return ' '.join(tokens)

# Demonstrate preprocessing on sample tweets
print("🔧 Text Preprocessing Demonstration:")
print("=" * 50)

sample_tweets = [
    "I LOVE this amazing movie!!! 😍😍😍 #amazing #movienight https://example.com @username",
    "This product is sooooo bad... I want my money back!!! 😡",
    "Can't believe how good this restaurant is! Best food ever! 🍕👌"
]

for i, tweet in enumerate(sample_tweets, 1):
    basic_processed = preprocess_text(tweet)
    advanced_processed = advanced_preprocess_text(tweet)
    
    print(f"\nExample {i}:")
    print(f"Original:    {tweet}")
    print(f"Basic:       {basic_processed}")
    print(f"Advanced:    {advanced_processed}")

print("\n" + "=" * 50)

In [ ]:
# Apply preprocessing to our dataset
print("🔄 Applying text preprocessing to dataset...")

# Apply advanced preprocessing
df['text_processed'] = df['text'].apply(advanced_preprocess_text)

# Remove any tweets that became empty after preprocessing
initial_count = len(df)
df = df[df['text_processed'].str.len() > 0]
final_count = len(df)

if initial_count != final_count:
    print(f"⚠️ Removed {initial_count - final_count} tweets that became empty after preprocessing")

print(f"✅ Preprocessing completed! {len(df)} tweets ready for analysis")

# Show before and after examples
print("\n📝 Before vs After Preprocessing Examples:")
sample_indices = df.head(3).index
for idx in sample_indices:
    print(f"\nOriginal:  {df.loc[idx, 'text']}")
    print(f"Processed: {df.loc[idx, 'text_processed']}")
    print(f"Sentiment: {df.loc[idx, 'sentiment']}")

# Check processed text statistics
df['processed_length'] = df['text_processed'].str.len()
df['processed_word_count'] = df['text_processed'].str.split().str.len()

print(f"\n📊 Processed Text Statistics:")
print(f"Average processed length: {df['processed_length'].mean():.1f}")
print(f"Average processed word count: {df['processed_word_count'].mean():.1f}")

## 5. Feature Extraction

Now we'll convert our preprocessed text into numerical features that machine learning algorithms can understand. We'll use TF-IDF (Term Frequency-Inverse Document Frequency) vectorization.

In [ ]:
# Initialize TF-IDF Vectorizer
print("🔢 Converting text to numerical features using TF-IDF...")

# We'll use TF-IDF with both unigrams and bigrams
tfidf_vectorizer = TfidfVectorizer(
    max_features=1000,  # Limit to top 1000 features for our small dataset
    ngram_range=(1, 2),  # Use both unigrams and bigrams
    min_df=1,  # Minimum document frequency (adjusted for small dataset)
    max_df=0.95,  # Maximum document frequency
    stop_words='english'  # Remove English stopwords
)

# Fit and transform the preprocessed text
X_tfidf = tfidf_vectorizer.fit_transform(df['text_processed'])

print(f"✅ TF-IDF Feature Matrix Shape: {X_tfidf.shape}")
print(f"📊 Number of unique features: {len(tfidf_vectorizer.get_feature_names_out())}")
print(f"📊 Matrix sparsity: {(1.0 - X_tfidf.nnz / (X_tfidf.shape[0] * X_tfidf.shape[1])):.3f}")

# Get the most important features (words)
feature_names = tfidf_vectorizer.get_feature_names_out()

# Calculate mean TF-IDF scores for each feature
mean_scores = np.array(X_tfidf.mean(axis=0)).flatten()

# Get top features
top_indices = mean_scores.argsort()[-20:][::-1]
top_features = [(feature_names[i], mean_scores[i]) for i in top_indices]

print(f"\n🔝 Top 10 TF-IDF Features:")
for feature, score in top_features[:10]:
    print(f"  {feature}: {score:.4f}")

# Also try CountVectorizer for comparison
print(f"\n🔢 Comparing with Count Vectorizer...")

count_vectorizer = CountVectorizer(
    max_features=1000,
    ngram_range=(1, 2),
    min_df=1,
    max_df=0.95,
    stop_words='english'
)

X_count = count_vectorizer.fit_transform(df['text_processed'])

print(f"✅ Count Vectorizer Matrix Shape: {X_count.shape}")
print(f"📊 Matrix sparsity: {(1.0 - X_count.nnz / (X_count.shape[0] * X_count.shape[1])):.3f}")

# We'll proceed with TF-IDF for our main analysis
X = X_tfidf
y = df['sentiment']

print(f"\n🎯 Final feature matrix: {X.shape}")
print(f"🎯 Target labels: {len(y)} samples")

## 6. Split Dataset for Training and Testing

Let's split our data into training and testing sets to properly evaluate our models.

In [ ]:
# Split the data into training and testing sets
print("📊 Splitting data into training and testing sets...")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.3,  # 30% for testing
    random_state=42,  # For reproducibility
    stratify=y  # Ensure balanced distribution across train/test
)

print(f"✅ Data split completed!")
print(f"📊 Training set: {X_train.shape[0]} samples")
print(f"📊 Testing set: {X_test.shape[0]} samples")

# Check sentiment distribution in train and test sets
print(f"\n🎯 Training set sentiment distribution:")
train_sentiment_dist = pd.Series(y_train).value_counts()
for sentiment, count in train_sentiment_dist.items():
    percentage = (count / len(y_train)) * 100
    print(f"  {sentiment.capitalize()}: {count} ({percentage:.1f}%)")

print(f"\n🎯 Testing set sentiment distribution:")
test_sentiment_dist = pd.Series(y_test).value_counts()
for sentiment, count in test_sentiment_dist.items():
    percentage = (count / len(y_test)) * 100
    print(f"  {sentiment.capitalize()}: {count} ({percentage:.1f}%)")

# Visualize the split
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Training set distribution
train_sentiment_dist.plot(kind='bar', ax=ax1, color=['#ff6b6b', '#4ecdc4', '#45b7d1'])
ax1.set_title('Training Set Sentiment Distribution', fontsize=12, fontweight='bold')
ax1.set_xlabel('Sentiment')
ax1.set_ylabel('Count')
ax1.tick_params(axis='x', rotation=45)

# Testing set distribution
test_sentiment_dist.plot(kind='bar', ax=ax2, color=['#ff6b6b', '#4ecdc4', '#45b7d1'])
ax2.set_title('Testing Set Sentiment Distribution', fontsize=12, fontweight='bold')
ax2.set_xlabel('Sentiment')
ax2.set_ylabel('Count')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print("✅ Ready for model training!")

## 7. Train Sentiment Classification Models

Now for the exciting part! We'll train multiple machine learning models and compare their performance.

In [ ]:
# Initialize multiple machine learning models
print("🤖 Training Multiple Machine Learning Models...")
print("=" * 50)

# Define our models
models = {
    'Naive Bayes': MultinomialNB(alpha=1.0),
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Support Vector Machine': SVC(random_state=42, probability=True),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42)
}

# Dictionary to store results
model_results = {}
trained_models = {}

# Train each model
for model_name, model in models.items():
    print(f"\n🔄 Training {model_name}...")
    
    # Train the model
    start_time = datetime.now()
    model.fit(X_train, y_train)
    training_time = (datetime.now() - start_time).total_seconds()
    
    # Make predictions
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test) if hasattr(model, 'predict_proba') else None
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    # Store results
    model_results[model_name] = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'training_time': training_time,
        'predictions': y_pred,
        'probabilities': y_pred_proba
    }
    
    trained_models[model_name] = model
    
    print(f"✅ {model_name} completed!")
    print(f"   Accuracy: {accuracy:.4f}")
    print(f"   F1-Score: {f1:.4f}")
    print(f"   Training time: {training_time:.2f} seconds")

print("\n🎉 All models trained successfully!")

## 8. Model Evaluation and Performance Metrics

Let's evaluate our models comprehensively and visualize their performance.

In [ ]:
# Create a comprehensive comparison of all models
print("📊 Model Comparison Summary:")
print("=" * 60)

# Create comparison DataFrame
comparison_data = []
for model_name, results in model_results.items():
    comparison_data.append({
        'Model': model_name,
        'Accuracy': results['accuracy'],
        'Precision': results['precision'],
        'Recall': results['recall'],
        'F1-Score': results['f1_score'],
        'Training Time (s)': results['training_time']
    })

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.round(4)

print(comparison_df.to_string(index=False))

# Find the best model
best_model_idx = comparison_df['F1-Score'].idxmax()
best_model_name = comparison_df.loc[best_model_idx, 'Model']
best_f1_score = comparison_df.loc[best_model_idx, 'F1-Score']

print(f"\n🏆 Best Model: {best_model_name} (F1-Score: {best_f1_score:.4f})")

# Visualize model comparison
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# Accuracy comparison
comparison_df.plot(x='Model', y='Accuracy', kind='bar', ax=ax1, color='skyblue', legend=False)
ax1.set_title('Model Accuracy Comparison', fontweight='bold')
ax1.set_ylabel('Accuracy')
ax1.tick_params(axis='x', rotation=45)
ax1.set_ylim(0, 1)

# F1-Score comparison
comparison_df.plot(x='Model', y='F1-Score', kind='bar', ax=ax2, color='lightgreen', legend=False)
ax2.set_title('Model F1-Score Comparison', fontweight='bold')
ax2.set_ylabel('F1-Score')
ax2.tick_params(axis='x', rotation=45)
ax2.set_ylim(0, 1)

# Precision vs Recall
ax3.scatter(comparison_df['Precision'], comparison_df['Recall'], s=100, alpha=0.7)
for i, model in enumerate(comparison_df['Model']):
    ax3.annotate(model, (comparison_df['Precision'].iloc[i], comparison_df['Recall'].iloc[i]), 
                xytext=(5, 5), textcoords='offset points')
ax3.set_xlabel('Precision')
ax3.set_ylabel('Recall')
ax3.set_title('Precision vs Recall', fontweight='bold')
ax3.grid(True, alpha=0.3)

# Training time comparison
comparison_df.plot(x='Model', y='Training Time (s)', kind='bar', ax=ax4, color='coral', legend=False)
ax4.set_title('Training Time Comparison', fontweight='bold')
ax4.set_ylabel('Training Time (seconds)')
ax4.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Generate detailed classification report for the best model
print(f"📋 Detailed Classification Report for {best_model_name}:")
print("=" * 50)

best_model = trained_models[best_model_name]
best_predictions = model_results[best_model_name]['predictions']

# Classification report
class_report = classification_report(y_test, best_predictions, output_dict=True)
print(classification_report(y_test, best_predictions))

# Confusion Matrix
print(f"\n📊 Confusion Matrix for {best_model_name}:")
cm = confusion_matrix(y_test, best_predictions)
sentiment_labels = sorted(df['sentiment'].unique())

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=sentiment_labels, yticklabels=sentiment_labels)
plt.title(f'Confusion Matrix - {best_model_name}', fontweight='bold')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.show()

# Calculate per-class metrics
print(f"\n📈 Per-Class Performance for {best_model_name}:")
for sentiment in sentiment_labels:
    precision = class_report[sentiment]['precision']
    recall = class_report[sentiment]['recall']
    f1 = class_report[sentiment]['f1-score']
    support = class_report[sentiment]['support']
    
    print(f"\n{sentiment.capitalize()}:")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall: {recall:.4f}")
    print(f"  F1-Score: {f1:.4f}")
    print(f"  Support: {support}")

# Cross-validation for more robust evaluation
print(f"\n🔄 Cross-Validation Results for {best_model_name}:")
cv_scores = cross_val_score(best_model, X_train, y_train, cv=5, scoring='f1_weighted')
print(f"CV F1-Scores: {cv_scores}")
print(f"Mean CV F1-Score: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

## 9. Predict Sentiments on New Tweets

Finally, let's create a prediction function and test our model on new, unseen tweets!

In [ ]:
def predict_sentiment(text, model, vectorizer, preprocessing_func):
    """
    Predict sentiment for a given text using the trained model
    """
    # Preprocess the text
    processed_text = preprocessing_func(text)
    
    # Transform using the fitted vectorizer
    text_features = vectorizer.transform([processed_text])
    
    # Make prediction
    prediction = model.predict(text_features)[0]
    
    # Get prediction probabilities
    if hasattr(model, 'predict_proba'):
        probabilities = model.predict_proba(text_features)[0]
        sentiment_labels = model.classes_
        prob_dict = dict(zip(sentiment_labels, probabilities))
    else:
        prob_dict = None
    
    return prediction, prob_dict

# Test on new tweets
test_tweets = [
    "This movie is absolutely fantastic! I loved every minute of it! 🎬✨",
    "Terrible service at this restaurant. Food was cold and staff was rude 😡",
    "The weather today is okay. Not too hot, not too cold.",
    "Just got my dream job! So excited to start this new chapter! 🎉",
    "My flight got cancelled again. This airline is the worst! ✈️😤",
    "Reading this book right now. It's decent, has some interesting parts.",
    "Best concert ever! The band was incredible! 🎵🔥",
    "This app keeps crashing. Very frustrating experience 📱",
    "Had lunch with friends today. Nice time, good food.",
    "Can't believe how amazing this vacation is! Paradise! 🏖️"
]

print("🔮 Predicting Sentiments on New Tweets:")
print("=" * 60)

# Use the best model for predictions
best_model = trained_models[best_model_name]

# Store results for analysis
prediction_results = []

for i, tweet in enumerate(test_tweets, 1):
    prediction, probabilities = predict_sentiment(tweet, best_model, tfidf_vectorizer, advanced_preprocess_text)
    
    print(f"\nTweet {i}:")
    print(f"Text: {tweet}")
    print(f"Predicted Sentiment: {prediction.upper()}")
    
    if probabilities:
        print("Confidence Scores:")
        for sentiment, prob in probabilities.items():
            emoji = "😞" if sentiment == 'negative' else "😐" if sentiment == 'neutral' else "😊"
            print(f"  {emoji} {sentiment.capitalize()}: {prob:.3f}")
    
    prediction_results.append({
        'tweet': tweet,
        'predicted_sentiment': prediction,
        'probabilities': probabilities
    })

print(f"\n✅ Completed predictions using {best_model_name}!")

In [ ]:
# Visualize prediction confidence
prediction_df = pd.DataFrame(prediction_results)

# Extract confidence scores for visualization
confidence_data = []
for idx, row in prediction_df.iterrows():
    if row['probabilities']:
        for sentiment, prob in row['probabilities'].items():
            confidence_data.append({
                'tweet_id': idx + 1,
                'sentiment': sentiment,
                'probability': prob,
                'is_predicted': sentiment == row['predicted_sentiment']
            })

confidence_df = pd.DataFrame(confidence_data)

# Plot confidence scores
plt.figure(figsize=(14, 8))

# Create a pivot table for heatmap
pivot_conf = confidence_df.pivot(index='tweet_id', columns='sentiment', values='probability')

# Create heatmap
sns.heatmap(pivot_conf, annot=True, fmt='.3f', cmap='RdYlBu_r', 
            cbar_kws={'label': 'Confidence Score'})
plt.title('Prediction Confidence Scores for Each Tweet', fontweight='bold', fontsize=14)
plt.xlabel('Sentiment')
plt.ylabel('Tweet ID')
plt.tight_layout()
plt.show()

# Summary of predictions
prediction_summary = prediction_df['predicted_sentiment'].value_counts()
print(f"\n📊 Prediction Summary:")
for sentiment, count in prediction_summary.items():
    percentage = (count / len(prediction_df)) * 100
    print(f"{sentiment.capitalize()}: {count} tweets ({percentage:.1f}%)")

# Interactive prediction function
print(f"\n🎮 Interactive Sentiment Analysis:")
print("You can now use the predict_sentiment function to analyze any text!")
print("Example usage:")
print("prediction, probabilities = predict_sentiment('Your text here', best_model, tfidf_vectorizer, advanced_preprocess_text)")

# Create a simple interface function
def analyze_text(text):
    """Simple interface for sentiment analysis"""
    prediction, probabilities = predict_sentiment(text, best_model, tfidf_vectorizer, advanced_preprocess_text)
    
    print(f"Text: {text}")
    print(f"Sentiment: {prediction.upper()}")
    if probabilities:
        print("Confidence:")
        for sentiment, prob in probabilities.items():
            print(f"  {sentiment.capitalize()}: {prob:.3f}")
    
    return prediction, probabilities

print(f"\nYou can also use: analyze_text('Your tweet here')")

## 🎉 Conclusion

Congratulations! You've successfully built a complete Twitter sentiment analysis system using NLP and machine learning. Let's summarize what we accomplished:

### 🏆 What We Built
1. **Data Processing Pipeline**: Loaded and cleaned Twitter data
2. **Text Preprocessing**: Comprehensive cleaning and normalization of tweet text
3. **Feature Extraction**: Converted text to numerical features using TF-IDF
4. **Model Training**: Trained and compared multiple ML algorithms
5. **Model Evaluation**: Comprehensive performance analysis with metrics and visualizations
6. **Prediction System**: Created a production-ready sentiment classifier

### 📊 Key Results
- **Best Model**: {best_model_name}
- **Accuracy**: {best_f1_score:.1%}
- **Capabilities**: Classifies text into positive, negative, and neutral sentiments

### 🚀 Next Steps
- **Scale Up**: Train with larger, real-world Twitter datasets
- **Deep Learning**: Experiment with neural networks (LSTM, BERT)
- **Real-time Analysis**: Connect to Twitter API for live sentiment monitoring
- **Multi-language**: Extend to support multiple languages
- **Deployment**: Deploy as a web service or mobile app

### 🛠 Tools Mastered
- **Pandas**: Data manipulation and analysis
- **Scikit-Learn**: Machine learning algorithms and evaluation
- **NLTK**: Natural language processing
- **Matplotlib/Seaborn**: Data visualization

You now have a solid foundation in NLP and sentiment analysis! 🎓